This example mirrors [tracts.dsi.html](https://niivue.com/demos/features/tracts.dsi.html).

In [ ]:
from pathlib import Path

from ipyniivue import download_dataset

BASE_API_URL = "https://niivue.com/demos/images"
DATA_FOLDER = Path("images")

# Download data for example
download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "mni152.nii.gz",
        "TR_S_R.tt.gz",
    ],
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

import ipyniivue

# Initialize
nv = ipyniivue.NiiVue()
nv.opts.back_color = [0, 0, 0, 1]
nv.opts.show_3d_crosshair = True
nv.opts.drag_mode = ipyniivue.DragMode.PAN
nv.opts.multiplanar_show_render = ipyniivue.ShowRender.ALWAYS
nv.opts.yoke_3d_to_2d_zoom = True

# Load Data
nv.load_volumes([{"path": DATA_FOLDER / "mni152.nii.gz"}])
nv.load_meshes([{"path": DATA_FOLDER / "TR_S_R.tt.gz", "rgba255": [0, 255, 255, 255]}])
nv.set_clip_plane(0.05, 0, 90)

# Widgets
fiber_occlusion = widgets.FloatSlider(min=0, max=100, value=0, description="Occlusion")
fiber_radius = widgets.FloatSlider(min=0, max=20, value=5, description="Radius")
fiber_dither = widgets.FloatSlider(min=0, max=10, value=0, description="Dither")
fiber_color = widgets.Dropdown(
    options=["Global", "Local", "Fixed"], value="Global", description="Fiber Color"
)
x_ray = widgets.FloatSlider(min=0, max=5, value=0, description="XRay")
shader_dropdown = widgets.Dropdown(
    options=nv.mesh_shader_names(), value="Specular", description="Shaders"
)


# Handlers
def on_occlusion_change(change):
    """Handle occlusion change."""
    if nv.meshes:
        nv.set_mesh_property(nv.meshes[0].id, "fiber_occlusion", change["new"] * 0.01)


def on_radius_change(change):
    """Handle radius change."""
    if nv.meshes:
        nv.set_mesh_property(nv.meshes[0].id, "fiber_radius", change["new"] * 0.1)


def on_dither_change(change):
    """Handle dither change."""
    if nv.meshes:
        nv.set_mesh_property(nv.meshes[0].id, "fiber_dither", change["new"] * 0.1)


def on_color_change(change):
    """Handle fiber color change."""
    if nv.meshes:
        nv.set_mesh_property(nv.meshes[0].id, "fiber_color", change["new"])


def on_xray_change(change):
    """Handle xray change."""
    nv.opts.mesh_xray = change["new"] / 20.0


def on_shader_change(change):
    """Handle shader change."""
    if nv.meshes:
        nv.set_mesh_shader(nv.meshes[0].id, change["new"])


# Bind Observers
fiber_occlusion.observe(on_occlusion_change, names="value")
fiber_radius.observe(on_radius_change, names="value")
fiber_dither.observe(on_dither_change, names="value")
fiber_color.observe(on_color_change, names="value")
x_ray.observe(on_xray_change, names="value")
shader_dropdown.observe(on_shader_change, names="value")

# Auto-set defaults
on_occlusion_change({"new": fiber_occlusion.value})
on_radius_change({"new": fiber_radius.value})
on_dither_change({"new": fiber_dither.value})
on_color_change({"new": fiber_color.value})
on_xray_change({"new": x_ray.value})
on_shader_change({"new": shader_dropdown.value})

# Layout
controls = widgets.VBox(
    [fiber_occlusion, fiber_radius, fiber_dither, fiber_color, x_ray, shader_dropdown]
)
display(widgets.VBox([controls, nv]))